In [ ]:
import nltk
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences

from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Embedding, GRU, SimpleRNN, RNN, LSTM, Flatten, Activation
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_fscore_support
# from baseline import bong
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import time


In [ ]:
df = pd.read_csv("/content/dontpatronizeme_pcl.tsv", 
                  sep='\t', 
                  names=["Id", "Art_id", "word","country", "Sentence", "power"],
                 skiprows = 3
                 )


In [ ]:
df.head()

,Id,Art_id,word,country,Sentence,power
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0
2,3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0
4,5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0


In [ ]:
len(df)

10469

In [ ]:
df.iloc[8639]

Id                8640
Art_id      @@16852855
word           migrant
country             ke
Sentence           NaN
power                0
Name: 8639, dtype: object

In [ ]:
dfna = df.isna()
ind = []
for i in range(len(df)):
  if dfna['Sentence'][i] == True:
    ind.append(i)

In [ ]:
ind

[8639]

In [ ]:
df = df.drop(ind)

In [ ]:
set_keywords = set(df['word'])

In [ ]:
len(set_keywords)

10

In [ ]:
list_of_sentences = np.array(df['Sentence'])
list_of_power = np.array(df['power'])

In [ ]:
len(list_of_sentences)

10468

In [ ]:
len(list_of_power)

10468

In [ ]:
for i in range(len(list_of_power)):
  if i in ind:
    continue
  if list_of_power[i]<=1:
    list_of_power[i] = 0
  else:
    list_of_power[i] = 1

In [ ]:
alphabet = {}
train_words = []
for sent in list_of_sentences:
    sent = sent.lower()
    words = []
    for char in sent:
      if char in alphabet:
        words.append(alphabet[char])
      else:
        alphabet[char] = len(alphabet) + 1
        words.append(alphabet[char])

    train_words.append(words)

In [ ]:
for sent in list_of_sentences[:5]:
  print(sent)

We 're living in times of absolute insanity , as I 'm pretty sure most people are aware . For a while , waking up every day to check the news seemed to carry with it the same feeling of panic and dread that action heroes probably face when they 're trying to decide whether to cut the blue or green wire on a ticking bomb -- except the bomb 's instructions long ago burned in a fire and imminent catastrophe seems the likeliest outcome . It 's hard to stay that on-edge for that long , though , so it 's natural for people to become inured to this constant chaos , to slump into a malaise of hopelessness and pessimism .
In Libya today , there are countless number of Ghanaian and Nigerian immigrants . These are the two countries with key macroeconomic challenges including unemployment . Let 's tackle this issue from the root and not the fruit . Thank you
White House press secretary Sean Spicer said that the focus of immigration actions would be illegal immigrants " who have also otherwise viol

In [ ]:
len(train_words)

10468

In [ ]:
alphabet

In [ ]:
len(alphabet)

58

In [ ]:
max_length = []
for x in list_of_sentences:
    max_length.append(len(x))
longest_sent = max(max_length)

In [ ]:
longest_sent

5493

In [ ]:
X = pad_sequences(train_words, padding='post', value=0, maxlen=longest_sent, truncating='post')

In [ ]:
X.shape

(10468, 5493)

In [ ]:
Y = np.reshape(list_of_power, (-1, 1))

In [ ]:
Y.shape

(10468, 1)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((8374, 5493), (8374, 1), (2094, 5493), (2094, 1))

In [ ]:
recurrent_model = Sequential()
recurrent_model.add(Embedding(input_dim=len(alphabet)+1, output_dim=64, input_length=longest_sent, mask_zero=False))
recurrent_model.add(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))
#recurrent_model.add(LSTM(units=64, return_sequences=True, recurrent_dropout=0.2))
# recurrent_model.add(Dropout(0.2))
recurrent_model.add(Flatten())

recurrent_model.add(Dense(units=1, activation='sigmoid'))
recurrent_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Set callback functions to early stop training and save the best model so far
es = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
recurrent_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5493, 64)          3776      
                                                                 
 lstm (LSTM)                 (None, 5493, 64)          33024     
                                                                 
 flatten (Flatten)           (None, 351552)            0         
                                                                 
 dense (Dense)               (None, 1)                 351553    
                                                                 
Total params: 388,353
Trainable params: 388,353
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train neural network
# recurrent_model.fit(train_x, train_y, validation_split=0.2, epochs=100, callbacks=[es], batch_size=64)
recurrent_model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[es], epochs=5, batch_size=216)

Epoch 1/5
39/39 [==============================] - 1207s 31s/step - loss: 0.3456 - accuracy: 0.8836 - val_loss: 0.3396 - val_accuracy: 0.9002
Epoch 2/5
39/39 [==============================] - 1201s 31s/step - loss: 0.3097 - accuracy: 0.9063 - val_loss: 0.3224 - val_accuracy: 0.9002
Epoch 3/5
39/39 [==============================] - 1208s 31s/step - loss: 0.2962 - accuracy: 0.9066 - val_loss: 0.3170 - val_accuracy: 0.9002
Epoch 4/5
39/39 [==============================] - 1200s 31s/step - loss: 0.2885 - accuracy: 0.9054 - val_loss: 0.3148 - val_accuracy: 0.9002
Epoch 5/5
39/39 [==============================] - 1199s 31s/step - loss: 0.2785 - accuracy: 0.9075 - val_loss: 0.3103 - val_accuracy: 0.8954


In [ ]:
y_test_pred = recurrent_model.predict(x_test)
print(y_test_pred)

[[0.0358189 ]
 [0.12320688]
 [0.04934735]
 ...
 [0.09054465]
 [0.25761607]
 [0.0903006 ]]


In [ ]:
y_pred = [0 if i < 0.5 else 1 for i in y_test_pred]

In [ ]:
y_pred.count(1)

14

In [ ]:
list(y_test).count(1)

209

In [ ]:
correct = [1 if y_pred[i] == y_test[i] else 0 for i in range(len(y_pred))]
print(sum(correct) / len(correct) * 100)

89.54154727793696


In [ ]:
f1 = f1_score(list(y_test), y_test_pred.round(), average='macro')
print(f1)

0.4813519642160384


In [ ]:
smaller_than = 0.499
list_of_numbers = list()

for y in y_test_pred:
    if y <= smaller_than:
        list_of_numbers.append(0)
    else:
        list_of_numbers.append(1)

#print(list_of_numbers)

test_y_new = []
for x in y_test:
    x_new = float(x)
    x_new = np.asarray(x_new)
    test_y_new.append(x_new)

In [ ]:
prec = precision_score(y_test, y_test_pred.round(), average='macro')
rec = recall_score(y_test, y_test_pred.round(), average='macro')
f1 = f1_score(y_test, y_test_pred.round(), average='macro')

print("Precision:", prec, "\n Recall:", rec, "\n F1-score:", f1)

Precision: 0.521668956043956 
 Recall: 0.5016016651225362 
 F1-score: 0.4813519642160384


In [ ]:
recurrent_model.save('nlp_lstm_weights.h5')

In [ ]:
import numpy as np